<a href="https://colab.research.google.com/github/tommylbro/DESKTOP-966UNPH/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [21]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-07-18 18:55:14--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  43.5MB/s    in 0.6s    

2025-07-18 18:55:15 (43.5 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [22]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [23]:
# add your code here - consider creating a new cell for each section of code
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# Load data
books = pd.read_csv('BX-Books.csv', encoding = "ISO-8859-1", sep=";", on_bad_lines='skip', engine='python')
ratings = pd.read_csv('BX-Book-Ratings.csv', encoding = "ISO-8859-1", sep=";", on_bad_lines='skip', engine='python')
users = pd.read_csv('BX-Users.csv', encoding = "ISO-8859-1", sep=";", on_bad_lines='skip', engine='python')

# Filter users with >=200 ratings
user_counts = ratings['User-ID'].value_counts()
ratings = ratings[ratings['User-ID'].isin(user_counts[user_counts >= 200].index)]

# Filter books with >=100 ratings
book_counts = ratings['ISBN'].value_counts()
ratings = ratings[ratings['ISBN'].isin(book_counts[book_counts >= 100].index)]

# Merge with books to get titles
ratings_with_books = ratings.merge(books, on='ISBN')

# Create pivot table (users as rows, books as columns)
book_pivot = ratings_with_books.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating').fillna(0)

# Sparse matrix for KNN
book_matrix = csr_matrix(book_pivot.values)

# Train KNN model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_matrix)

# Function to get recommendations
def get_recommends(book_title):
    if book_title not in book_pivot.index:
        return f"Book '{book_title}' not found in the dataset after filtering."

    book_index = book_pivot.index.get_loc(book_title)
    distances, indices = model.kneighbors(book_pivot.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_books.append([book_pivot.index[indices.flatten()[i]], distances.flatten()[i]])

    return [book_title, recommended_books]

In [24]:
# function to return recommended books - this will be tested

In [25]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books_titles = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  # Check the top 4 recommended books
  for i in range(4):
    if recommends[1][i][0] not in recommended_books_titles:
      test_pass = False
    # Find the corresponding distance in the expected list
    try:
        expected_dist_index = recommended_books_titles.index(recommends[1][i][0])
        if abs(recommends[1][i][1] - recommended_books_dist[expected_dist_index]) >= 0.05:
            test_pass = False
    except ValueError:
        # This means the recommended book title was not found in the expected list,
        # which is already covered by the check above, but good to have for clarity.
        test_pass = False

  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', np.float64(0.7234864549790632)], ["The Pilot's Wife : A Novel", np.float64(0.8192678336213406)], ['The Joy Luck Club', np.float64(0.8198604785739199)], ['The Notebook', np.float64(0.8236682900571164)], ['Bel Canto: A Novel', np.float64(0.8247874880681116)]]]
You haven't passed yet. Keep trying!
